In [2]:
import pandas as pd
import requests

In [3]:
def extract_mars_weather_data(api_key):
    try: 
        with requests.Session() as session:
            response = session.get(f"https://api.nasa.gov/insight_weather/?api_key={api_key}&feedtype=json&ver=1.0")
            response.raise_for_status()
            data = response.json()

        # Достаем только дни (сола) 
        sol_keys = data.get("sol_keys", [])
        return data, sol_keys
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при загрузке данных из API: {e}")
        return None, None

In [4]:
def get_mars_weather_data_to_DataFrame(data, sol_keys):
    try:
        if not data or not sol_keys:
            print("Нет данных для создания DataFrame")
        weather_data = [data[sol] for sol in sol_keys]
        return pd.DataFrame(weather_data)
    except Exception as e:
        print(f"Ошбика при получении данных: {e}")
        return None

In [5]:
def get_temperature_data(data, sol_keys):
    try:
        if not data or not sol_keys:
            print("Нет данных о температуре")
        temperature_data = []
        for sol in sol_keys:
            sol_data = data[sol]  # Данные за конкретный sol
            if 'AT' in sol_data:  # Проверяем, есть ли данные о температуре
                temperature_data.append({
                    'sol': sol,
                    'min_temp': sol_data['AT']['mn'],  # Минимальная температура
                    'max_temp': sol_data['AT']['mx'],  # Максимальная температура
                    'avg_temp': sol_data['AT']['av']   # Средняя температура
                })

        return pd.DataFrame(temperature_data)
    except Exception as e:
        print(f"Ошбика при получении данных о температуре: {e}")
        return None

In [6]:
def get_min_temperature(df):
    return df['min_temp'].min() if not df.empty else None

In [7]:
def get_max_temperature(df):
    return df['max_temp'].max() if not df.empty else None


In [8]:
def get_pressure_data(data, sol_keys):
    try:
        if not data or not sol_keys:
            print("Нет данных о давлении")
        pressure_data = []
        for sol in sol_keys:
            sol_data = data[sol]  # Данные за конкретный sol
            if 'PRE' in sol_data:  # Проверяем, есть ли данные о давлении
                pressure_data.append({
                    'sol': sol,
                    'min_pres': sol_data['PRE']['mn'],  # Минимальное давление
                    'max_pres': sol_data['PRE']['mx'],  # Максимальное давление
                    'avg_pres': sol_data['PRE']['av']   # Среднее давление
                })

        return pd.DataFrame(pressure_data)
    except Exception as e:
        print(f"Ошбика при получении данных о давлении: {e}")
        return None

In [9]:
def get_avg_pressure(df):
    return df['avg_pres'].mean() if not df.empty else None
    

In [10]:
def write_mars_weather_to_excel(mars_weather_df, temperature_data, pressure_data, filename):
    try:
        with pd.ExcelWriter(filename, engine="xlsxwriter") as writer:
            if mars_weather_df is not None:
                mars_weather_df.to_excel(writer, sheet_name="Mars_weather", index=False)
            if temperature_data is not None:
                temperature_data.to_excel(writer, sheet_name="Temperature_data", index=False)
            if pressure_data is not None:
                pressure_data.to_excel(writer, sheet_name="Pressure_data", index=False)
        print(f"Данные успешно сохранены в '{filename}'")
    except Exception as e:
        print(f"Ошибка записи данных в Excel: {e}")

In [11]:
def write_analyze_data_to_csv(min_temp, max_temp, avg_pressure, filename):
    try:
        data_to_csv = {
            "Data": ['Min temperature', 'Max temperature', 'Average Pressure'],
            "Value": [min_temp, max_temp, avg_pressure]
        }
        analyze_data = pd.DataFrame(data_to_csv)
        analyze_data.to_csv(filename, index=False)
        print(f"Данные успешно загружены в {filename}")
    except Exception as e:
        print(f"Ошибка записи данных в CSV: {e}")


In [12]:
def main(api_key):
    mars_weather, sol_keys = extract_mars_weather_data(api_key)
    
    if mars_weather is None or sol_keys is None:
        print("Не удалось загрузить данные.")
        return
    
    mars_weather_df = get_mars_weather_data_to_DataFrame(mars_weather, sol_keys)
    temperature_data = get_temperature_data(mars_weather, sol_keys)
    pressure_data = get_pressure_data(mars_weather, sol_keys)
    min_temp = get_min_temperature(temperature_data)
    max_temp = get_max_temperature(temperature_data)
    avg_pressure = get_avg_pressure(pressure_data)
    write_mars_weather_to_excel(mars_weather_df, temperature_data, pressure_data, "files/mars_weather.xlsx")
    write_analyze_data_to_csv(min_temp, max_temp, avg_pressure, 'files/mars_analyze_data.csv')


In [13]:
if __name__ == "__main__":
    API_KEY = "502SJWBgC67LUsXow8S2uPQtZC29d2iDQ7Og6Hkg"
    main(API_KEY)


Данные успешно сохранены в 'files/mars_weather.xlsx'
Данные успешно загружены в files/mars_analyze_data.csv
